# Trend Factor Analysis

This notebook walks through a trend factor analysis, for both time series and cross-sectional strategies, in digital assets.

We will construct various trend factors and compare their performance across a range of statistical tests and financial evaluation criteria.

Trend following is a ubiquitous factor investing strategy and remains one of the best ways to capture convexity in cryptoassets.

To conduct this analysis, we will use **FactorLab**, an open-source python package we have created specifically for alpha and risk factor analysis. 

To install **FactorLab**:
`pip install factorlab`

In [1]:
# uncomment to install factorlab
# pip install factorlab

In [2]:
import pandas as pd
import numpy as np

from cryptodatapy.extract.datarequest import DataRequest
from cryptodatapy.extract.getdata import GetData
from cryptodatapy.transform.clean import CleanData

from factorlab.factors.trend import Trend
from factorlab.feature_engineering.transform import Transform
from factorlab.feature_analysis.factor_analysis import Factor
from factorlab.feature_analysis.param_grid_search import *
from factorlab.feature_analysis.time_series_analysis import linear_reg
from factorlab.feature_analysis.performance import Performance
from factorlab.data_viz.data_viz import plot_series, plot_bar, plot_table

## 1. Data 

To collect the necessary data for this analysis (perpetual futures, funding rates, spot prices, and aggregate spot prices), we use **CryptoDataPy**, an open source python library that makes it easy to build high quality data pipelines for the analysis of cryptoassets.

With it, we can pull data from various exchanges and data vendors with ease, clean and stich it to create the longest possible price and total return series.

To install **CryptoDataPy**:
`pip install cryptodatapy`

In [3]:
# uncomment to install cryptodatapy
# pip install cryptodatapy

### 1.1 Collect Data

- Funding rates and perptual futures OHLC prices from **Binance** futures exchange (2019-present)
- OHLC spot prices from **Binance** spot exchange (2017-2019)
- OHLC spot prices from **Cryptocompare** (2010-2017)

#### Binance

In [ ]:
# get all Binance perp futures tickers
data_req = DataRequest(source='ccxt')
perp_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binanceusdm', as_list=True)

In [ ]:
# get Binance spot tickers
data_req = DataRequest(source='ccxt')
spot_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binance', as_list=True)

In [ ]:
# find intersecting tickers
binance_tickers = [ticker for ticker in fut_tickers if ticker in spot_tickers]

In [ ]:
# number of tickers
len(binance_tickers)

#### Cryptocompare 

In [ ]:
# get cryptocompare tickers
data_req = DataRequest(source='cryptocompare')
cc_tickers = GetData(data_req).get_meta(method='get_assets_info', as_list=True)

In [ ]:
# keep only USDT ticker
bin_tickers = []
for ticker in binance_tickers:
    if '/' in ticker and ticker.split('/')[1] == 'USDT':
        bin_tickers.append(ticker.split('/')[0])

In [ ]:
# usdt tickers
usdt_tickers = [ticker.split('/')[0] for ticker in binance_tickers if '/'in ticker and ticker.split('/')[1] == 'USDT']

In [ ]:
# intersecting tickers
tickers = [ticker for ticker in usdt_tickers if ticker in cc_tickers]

#### Perpetual Futures

In [ ]:
# pull daily OHLC and funding rates for perp futures on Binance USDM exchange
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume', 'funding_rate'], 
                       mkt_type='perpetual_future', 
                       freq='d')

In [ ]:
df1 = GetData(data_req).get_series()

In [ ]:
df1.head()

In [ ]:
df1.to_csv('factorlab/src/factorlab/datasets/data/binance_perp_fut_prices.csv')

#### Spot Prices

In [ ]:
# pull OHLC from Binance
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [ ]:
df2 = GetData(data_req).get_series()

In [ ]:
df2.head()

In [ ]:
df2.to_csv('factorlab/src/factorlab/datasets/data/binance_spot_prices.csv')

#### Historical Spot

In [ ]:
# pull close and funding rates for agg spot data from CryptoCompare
data_req = DataRequest(source='cryptocompare',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [ ]:
df3 = GetData(data_req).get_series()

In [ ]:
df3.head()

In [ ]:
df3.to_csv('factorlab/src/factorlab/datasets/data/cc_spot_prices.csv')

In [ ]:
# # pull csv perp futures prices
# df1 = pd.read_csv('factorlab/src/factorlab/datasets/data/binance_perp_fut_prices.csv', 
#                   index_col=['date', 'ticker'],
#                   parse_dates=True
#                  )

In [ ]:
# # pull csv spot prices, binance
# df2 = pd.read_csv('factorlab/src/factorlab/datasets/data/binance_spot_prices.csv', 
#                   index_col=['date', 'ticker'],
#                   parse_dates=True
#                  )

In [ ]:
# # pull csv spot prices, cryptocompare
# df3 = pd.read_csv('factorlab/src/factorlab/datasets/data/cc_spot_prices.csv', 
#                   index_col=['date', 'ticker'],
#                   parse_dates=True
#                  )

In [ ]:
# stich dfs to extend price time series
df = df1.combine_first(df2).combine_first(df3)
df = df[['open', 'high', 'low', 'close', 'volume', 'funding_rate']]
df.funding_rate = df.funding_rate.fillna(0)

##  1.2 Clean Data

Now, that we have all of our raw data, we can clean our data using the *CleanData* class in **CryptoDataPy** and chaining the following methods:

- *Filter outliers* to remove outliers using interquantile range and a threshold.
- *Repair outliers* and missing values using the IQR expected value.
- *Filter average trading value* to reduce the trading universe to include only asset trading a minimum of daily dollar volume (1 mil USD).
- *Remove missing values gaps* to drop the values of any series where sequential missing values are above some threshodl value.

In [ ]:
# plot close series
df.unstack().close.plot(subplots=True, sharex=False, figsize=(15,500));

In [ ]:
# drop tickers with nobs < ts_obs
obs = df.groupby(level=1).count().min(axis=1)
drop_tickers_list = obs[obs < 365].index.to_list()
df = df.drop(drop_tickers_list, level=1, axis=0)

# drop tickers with nobs < cs_obs
obs = df.groupby(level=0).count().min(axis=1)
idx_start = obs[obs > 3].index[0]
df = df.unstack()[df.unstack().index > idx_start].stack()

In [ ]:
# # clean data, but do not include the funding rate data in the object since outliers tend to be correct values
# clean_df = CleanData(df.drop(columns='funding_rate')).\
#                    filter_outliers(od_method='iqr', thresh_val=10).\
#                    repair_outliers(imp_method='fcst').\
#                    filter_avg_trading_val(thresh_val=1000000).\
#                    filter_min_nobs(ts_obs=365, cs_obs=3).\
#                    filter_missing_vals_gaps(gap_window=30).get(attr='df')

In [ ]:
# # concat funding rate data
# clean_df = pd.concat([clean_df, df[['volume', 'funding_rate']]], axis=1).dropna()

In [ ]:
# clean_df.tail()

In [ ]:
# number of assets
len(list(df.index.droplevel(0).unique()))

In [ ]:
# # plot data
# clean_df.unstack().close.plot(subplots=True, sharex=False, figsize=(15,250));

## 1- Factor and Target Construction

In [ ]:
# create ohlc df
ohlc = df[['open', 'high', 'low', 'close']].copy()

### 1.1 - Create Targets
Create log return and forward return target variables.

In [ ]:
# compute total returns
ret_df = pd.concat([Transform(df.loc[:,:'close']).returns(), df[['volume', 'funding_rate']]], join='inner', axis=1)
tr_df = ret_df.loc[:,:'close'].subtract(ret_df.funding_rate, axis=0)
tr_df = pd.concat([tr_df, df[['volume', 'funding_rate']]], join='inner', axis=1)

In [ ]:
# compute forward returns
fwd_spot_ret = Transform(df.close).returns(lags=1, forward=True).close.to_frame('fwd_ret_1')
fwd_spot_ret['fwd_ret_5'] = Transform(df.close).returns(lags=5, forward=True)
fwd_spot_ret['fwd_ret_7'] = Transform(df.close).returns(lags=7, forward=True)
fwd_spot_ret['fwd_ret_10'] = Transform(df.close).returns(lags=10, forward=True)
fwd_spot_ret['fwd_ret_14'] = Transform(df.close).returns(lags=14, forward=True)
fwd_spot_ret['fwd_ret_20'] = Transform(df.close).returns(lags=20, forward=True)
fwd_spot_ret['fwd_ret_30'] = Transform(df.close).returns(lags=30, forward=True)
fwd_spot_ret['fwd_ret_60'] = Transform(df.close).returns(lags=60, forward=True)
fwd_spot_ret['fwd_ret_90'] = Transform(df.close).returns(lags=90, forward=True)
fwd_spot_ret['fwd_ret_180'] = Transform(df.close).returns(lags=180, forward=True)
fwd_spot_ret['fwd_ret_365'] = Transform(df.close).returns(lags=365, forward=True)

In [ ]:
# normalize fwd rets
fwd_ret_norm = Transform(fwd_spot_ret).normalize_ts(window_type='expanding')

In [ ]:
# compute fwd relative returns
fwd_rel_ret = fwd_spot_ret.copy()
for col in fwd_rel_ret.columns:
    fwd_rel_ret[col] = fwd_spot_ret[col] - fwd_spot_ret[col].groupby('date').mean()

In [ ]:
# mkt portfolio returns
mkt_ret = ret_df.close.groupby('date').mean().to_frame('mkt_ret')

### 1.2. Compute Trend Factors

- We compute ~ a dozen trend factors across which we will compare performance.

In [ ]:
# trend factors
# price mom
trend_df = Trend(ohlc, vwap=True, log=True, lookback=5).price_mom().rename(columns={"vwap": "price_mom_5"})
trend_df['price_mom_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).price_mom()
trend_df['price_mom_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).price_mom()
trend_df['price_mom_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).price_mom()
trend_df['price_mom_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).price_mom()
trend_df['price_mom_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).price_mom()
trend_df['price_mom_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).price_mom()
trend_df['price_mom_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).price_mom()
# moving window diff
trend_df['mw_diff_5'] = Trend(ohlc, vwap=True, log=True, lookback=5, smoothing='ewm').mw_diff()
trend_df['mw_diff_10'] = Trend(ohlc, vwap=True, log=True, lookback=10, smoothing='ewm').mw_diff()
trend_df['mw_diff_15'] = Trend(ohlc, vwap=True, log=True, lookback=15, smoothing='ewm').mw_diff()
trend_df['mw_diff_30'] = Trend(ohlc, vwap=True, log=True, lookback=30, smoothing='ewm').mw_diff()
trend_df['mw_diff_45'] = Trend(ohlc, vwap=True, log=True, lookback=45, smoothing='ewm').mw_diff()
trend_df['mw_diff_90'] = Trend(ohlc, vwap=True, log=True, lookback=90, smoothing='ewm').mw_diff()
trend_df['mw_diff_180'] = Trend(ohlc, vwap=True, log=True, lookback=180, smoothing='ewm').mw_diff()
trend_df['mw_diff_365'] = Trend(ohlc, vwap=True, log=True, lookback=365, smoothing='ewm').mw_diff()
# divergence
trend_df['divergence_5'] = Trend(ohlc, vwap=True, log=True, lookback=5, smoothing='ewm').divergence()
trend_df['divergence_10'] = Trend(ohlc, vwap=True, log=True, lookback=10, smoothing='ewm').divergence()
trend_df['divergence_15'] = Trend(ohlc, vwap=True, log=True, lookback=15, smoothing='ewm').divergence()
trend_df['divergence_30'] = Trend(ohlc, vwap=True, log=True, lookback=30, smoothing='ewm').divergence()
trend_df['divergence_45'] = Trend(ohlc, vwap=True, log=True, lookback=45, smoothing='ewm').divergence()
trend_df['divergence_90'] = Trend(ohlc, vwap=True, log=True, lookback=90, smoothing='ewm').divergence()
trend_df['divergence_180'] = Trend(ohlc, vwap=True, log=True, lookback=180, smoothing='ewm').divergence()
trend_df['divergence_365'] = Trend(ohlc, vwap=True, log=True, lookback=365, smoothing='ewm').divergence()
# exp weighted mov avg crossover
trend_df['ewma_xover_2'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[2, 4, 8], l_k=[6, 12, 24], signal=True)
trend_df['ewma_xover_3'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[3, 6, 12], l_k=[9, 18, 36], signal=True)
trend_df['ewma_xover_4'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[4, 8, 16], l_k=[12, 24, 48], signal=True)
trend_df['ewma_xover_5'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[5, 10, 20], l_k=[15, 30, 60], signal=True)
trend_df['ewma_xover_6'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[6, 12, 24], l_k=[18, 36, 72], signal=True)
trend_df['ewma_xover_7'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[7, 14, 28], l_k=[21, 42, 84], signal=True)
trend_df['ewma_xover_ahl'] = Trend(ohlc, vwap=True, log=True,).ewma_wxover(s_k=[8, 16, 32], l_k=[24, 48, 96], signal=True)
# rsi
trend_df['rsi_5'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=5).rsi()
trend_df['rsi_10'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=10).rsi()
trend_df['rsi_15'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=15).rsi()
trend_df['rsi_30'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=30).rsi()
trend_df['rsi_45'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=45).rsi()
trend_df['rsi_90'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=90).rsi()
trend_df['rsi_180'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=180).rsi()
trend_df['rsi_365'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=365).rsi()
# stochastic
trend_df['stoch_5'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=5).stochastic()
trend_df['stoch_10'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=10).stochastic()
trend_df['stoch_15'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=15).stochastic()
trend_df['stoch_30'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=30).stochastic()
trend_df['stoch_45'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=45).stochastic()
trend_df['stoch_90'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=90).stochastic()
trend_df['stoch_180'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=180).stochastic()
trend_df['stoch_365'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=365).stochastic()
# intensity
trend_df['intensity_5'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=5).intensity()
trend_df['intensity_10'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=10).intensity()
trend_df['intensity_15'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=15).intensity()
trend_df['intensity_30'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=30).intensity()
trend_df['intensity_45'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=45).intensity()
trend_df['intensity_90'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=90).intensity()
trend_df['intensity_180'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=180).intensity()
trend_df['intensity_365'] = Trend(ohlc, vwap=True, log=True, smoothing='ewm', lookback=365).intensity()
# time trend
trend_df['time_trend_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).time_trend()
trend_df['time_trend_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).time_trend()
trend_df['time_trend_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).time_trend()
trend_df['time_trend_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).time_trend()
trend_df['time_trend_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).time_trend()
trend_df['time_trend_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).time_trend()
trend_df['time_trend_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).time_trend()
trend_df['time_trend_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).time_trend()
# alpha mom
trend_df['alpha_mom_5'] = Trend(ohlc, vwap=True, log=True, lookback=5).alpha_mom()
trend_df['alpha_mom_10'] = Trend(ohlc, vwap=True, log=True, lookback=10).alpha_mom()
trend_df['alpha_mom_15'] = Trend(ohlc, vwap=True, log=True, lookback=15).alpha_mom()
trend_df['alpha_mom_30'] = Trend(ohlc, vwap=True, log=True, lookback=30).alpha_mom()
trend_df['alpha_mom_45'] = Trend(ohlc, vwap=True, log=True, lookback=45).alpha_mom()
trend_df['alpha_mom_90'] = Trend(ohlc, vwap=True, log=True, lookback=90).alpha_mom()
trend_df['alpha_mom_180'] = Trend(ohlc, vwap=True, log=True, lookback=180).alpha_mom()
trend_df['alpha_mom_365'] = Trend(ohlc, vwap=True, log=True, lookback=365).alpha_mom()

In [ ]:
# normalize trend factors
trend_z_df = Transform(trend_df).normalize_ts(window_type='expanding')

# Factor Analysis

## 1- Statistical Tests

Statistical tests allow us to assess the strenght of the relationship between the alpha factors (features) and foward returns (target variable).


### 1.1. Historical Correlation and Association Measures
Correlation measures, e.g. spearman rank correlation (aka information coefficient), and association measures, e.g. mutul information, allow us to evaluate the predictive relationship between an alpha factors and forward returns. 
- The `strategy` parameter in the `filter` method can be set to either "ts" or "cs" for either time series or cross-sectional strategies.

In [ ]:
# compute stats for time series strategies
Factor(trend_df, fwd_ret_norm.fwd_ret_1, strategy='ts', factor_bins=5, target_bins=3, window_type='fixed').filter(rank_on='spearman_r')

In [ ]:
# compute stats for cross-sectional carry strategies 
Factor(trend_df, fwd_ret_norm.fwd_ret_1, strategy='cs', factor_bins=5, target_bins=3, window_type='fixed').filter(rank_on='spearman_r')

### 1.2 - Moving Window IC
The information coefficient (or spearman rank correlation) can be computed over a rolling window to allow us to see the change in predictive relationship between the factors and forward returns.


In [ ]:
# IC rolling window for time series strategy
ic_ts_df = Factor(trend_df, ret_df.close, strategy='ts_ls', window_size=365).ic(factor='price_mom_30')

In [ ]:
plot_series(ic_ts_df.dropna(), 
            title='Information Coefficient',
            subtitle='Time series, 365 day rolling window'
           )

In [ ]:
# IC rolling window for time series strategy
ic_cs_df = Factor(trend_df, ret_df.close, strategy='cs_ls', window_size=365).ic(factor='price_mom_30')

In [ ]:
plot_series(ic_cs_df.dropna(), 
            title='Information Coefficient',
            subtitle='Cross-sectional, 365 day rolling window'
           )

### 1.3 - Regression
We can regress factors on forward returns (normalized) to assess their economic and statistical significance.


In [ ]:
# pooled regression for time series 
Factor(trend_z_df, fwd_ret_norm.fwd_ret_7, strategy='ts').regression(method='pooled')

In [ ]:
# cross sectional fama-macbeth
Factor(trend_z_df, fwd_ret_norm.fwd_ret_7, strategy='cs').regression(method='fama-macbeth')

## 2- Factor Returns
Factor returns (net of t-cost estimate) provide a measure of factor performance. 
- The `strategy` parameter in the factor returns function allows you to explore factor returns for both long-only '_l' and long/short strategies '_ls', for either time series 'ts' or cross-sectional 'cs' implementations. 
- The `tails` parameter allows you to keep only factor values in the tails of the distribution and ignore the rest. Often, the most predictive information is contained in the tails.

### 2.1 - Time Series

In [ ]:
# ts trend returns
trend_ts_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='ts_ls', factor_bins=3).returns(signal_type='signal')

In [ ]:
trend_ts_perf_table = Performance(trend_ts_ret, mkt_ret=mkt_ret, ret_type='log').table(metrics='all', rank_on='Sharpe ratio').iloc[:20]

In [ ]:
trend_ts_perf_table

In [ ]:
Performance(trend_ts_ret, mkt_ret=mkt_ret, ret_type='log').plot_metric()

### 2.2 - Cross-Sectional
Cross-sectional strategies sorts assets in the cross-sectiona into equal-weighted portfolios based on their factor values. Weights to each portfolio can be scaled by on the portfolios rank (e.g. 1 for the assets in the top quintile of factor values, 0.5 for the 2nd quintile, etc) or top vs. bottom ranking are given values of 1 and -1.

In [ ]:
# cs trend returns
trend_cs_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='cs_ls', factor_bins=5).returns(signal_type='signal_quantiles', tails='two', rebalancing=7, t_cost=0.0025, weighting='ew')

In [ ]:
trend_cs_perf_table = Performance(trend_cs_ret, mkt_ret=mkt_ret, ret_type='log').table(metrics='all', rank_on='Sharpe ratio').iloc[:20]

In [ ]:
trend_cs_perf_table

In [ ]:
Performance(trend_cs_ret, mkt_ret=mkt_ret, ret_type='log').plot_metric()

## 3- Quantile Returns
Factor quantile returns provide a measure of alpha factor performance. 

We can use the ```quantiles``` method to analyze returns across bins/quantiles in order to assess the robustness of the alpha factor. Returns which increase monotonically with quantiles are likely to be more reliable that those with positive but non-monotonic relationships.

- The `factor` parameter in the quantile method allows us to select the factor for which to create quantiles. 
- The `metric` parameter allows us to compute a specific metrics from the ```Performance``` class. Here we use the default 'ret' value for returns.
- The ```rebalancing``` parameter allows us to modify the rebalancing frequency for the strategy. This value defaults to 1, meaning changing with each period/frequency. We select '7' for weekly rebalancing with daily data frequency.

### Time Series Strategy

In [ ]:
trend_ts_quantile_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='ts_ls', factor_bins=3).quantiles(factor=trend_ts_perf_table.iloc[0].name, metric='ret', rebalancing=7)

In [ ]:
plot_series(trend_ts_quantile_ret.cumsum(),
            title="Time Series Trend Strategy",
            subtitle=f"Factor: {trend_ts_perf_table.iloc[0].name}",
            y_label='Cumulative Log Returns'
           )

In [ ]:
trend_ts_quantile_ann_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='ts_ls', factor_bins=3).quantiles(factor=trend_ts_perf_table.iloc[0].name, metric='ann_ret', rebalancing=7)

In [ ]:
plot_bar(trend_ts_quantile_ann_ret,
            title="Time Series Trend Strategy",
            subtitle=f"Factor: {trend_ts_perf_table.iloc[0].name}",
            y_label='Annual Returns'
           )

In [ ]:
trend_ts_quantile_vol = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='ts_ls', factor_bins=3).quantiles(factor=trend_ts_perf_table.iloc[0].name, metric='ann_vol', rebalancing=7)

In [ ]:
plot_bar(trend_ts_quantile_vol,
            title="Time Series Trend Strategy",
            subtitle=f"Factor: {trend_ts_perf_table.iloc[0].name}",
            y_label='Annual Volatility'
        )

In [ ]:
trend_ts_quantile_sr = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='ts_ls', factor_bins=3).quantiles(factor=trend_ts_perf_table.iloc[0].name, metric='sharpe_ratio', rebalancing=7)

In [ ]:
plot_bar(trend_ts_quantile_sr,
            title="Time Series Trend Strategy",
            subtitle=f"Factor: {trend_ts_perf_table.iloc[0].name}",
            y_label='Sharpe Ratio'
        )

### Cross-sectional Strategy

In [ ]:
trend_cs_quantile_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='cs_ls', factor_bins=5).quantiles(factor=trend_cs_perf_table.iloc[0].name, metric='ret', rebalancing=7)

In [ ]:
plot_series(trend_cs_quantile_ret.cumsum(),
            title="Cross-sectional Trend Strategy",
            subtitle=f"Factor: {trend_cs_perf_table.iloc[0].name}",
            y_label='Log of Cumulative Returns'
           )

In [ ]:
trend_cs_quantile_ann_ret = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='cs_ls', factor_bins=5).quantiles(factor=trend_cs_perf_table.iloc[0].name, metric='ann_ret', rebalancing=7)

In [ ]:
plot_bar(trend_cs_quantile_ann_ret,
            title="Cross-sectional Trend Strategy",
            subtitle=f"Factor: {trend_cs_perf_table.iloc[0].name}",
            y_label='Annual Returns'
           )

In [ ]:
trend_cs_quantile_vol = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='cs_ls', factor_bins=5).quantiles(factor=trend_cs_perf_table.iloc[0].name, metric='ann_vol', rebalancing=7)

In [ ]:
plot_bar(trend_cs_quantile_vol,
            title="Cross-sectional Trend Strategy",
            subtitle=f"Factor: {trend_cs_perf_table.iloc[0].name}",
            y_label='Annual Volatility'
        )

In [ ]:
trend_cs_quantile_sr = Factor(trend_df.loc['2016-01-01':], tr_df.close, strategy='cs_ls', factor_bins=5).quantiles(factor=trend_cs_perf_table.iloc[0].name, metric='sharpe_ratio', rebalancing=7)

In [ ]:
plot_bar(trend_cs_quantile_sr,
            title="Cross-sectional Trend Strategy",
            subtitle=f"Factor: {trend_cs_perf_table.iloc[0].name}",
            y_label='Sharpe Ratio'
        )

## 4- Factor Robustness

A robust factor should produce positive returns regardless of changes in construction methodology, inputs and parameter values. One way of assessing the robustness of a factor is to examine how returns change across various implementations, inputs, parameters and sample periods. This can be done as part with the ```factor_param_grid_search``` function:
- The ```feat_args``` parameters allows us to vary all of the attributes that are used to compute all factors with the ```Trend``` class.
- The ```algo_args``` parameters allow us to vary all of the arguments used by the methods which compute specific trend algorithms, e.g. ```price_mom``` or ```time_trend```.

### 4.1 Factor Parameter Grid Search

#### Time Series
We explore the parameter space for the best performing time series trend strategy.

In [ ]:
metrics_df = factor_param_grid_search(df, tr_df.close, Trend, 'stochastic', metric='sharpe_ratio',
                                             feat_args={
                                                 'smoothing': ['smw', 'ewm', 'median'],
                                                 'lookback': [5, 10, 15, 30, 60, 90, 120, 180, 365],
                                             },
                                             algo_args={'stochastic': ['k', 'd'],
                                                        'signal': [True, False],
                                                        },
                                             factor_args={'strategy': 'ts_ls'},
                                             ret_args={'signal_type': 'signal', 'rebalancing': 7, 
                                                       't_cost': 0.0025, 'weighting': 'ew'}
                                             )

In [ ]:
param_matrix = param_heatmap(metrics_df.dropna(), 
                             metric='sharpe_ratio', 
                             fixed_params={'signal': True, 'smoothing': 'ewm'}, 
                             plot_params=['stochastic', 'lookback']
                            )

### 4.1 Strategy Parameter Grid Search

The ```Factor``` class has a ```return``` method which computes factor returns. The class attributes and method arguments can be optimized to achieve better performance. We can use the ```strategy_param_grid_search``` to assess the factor strategy's sensitivity to these inputs.

#### Cross Sectional

We explore the parameter space for the best performing cross-sectional trend strategy.

In [ ]:
strat_df = strategy_param_grid_search(trend_df[trend_cs_perf_table.iloc[0].name], ret_df.close, metric='sharpe_ratio',                         
                                
                                factor_args = {
                                    'strategy': ['cs_ls'],
                                     'factor_bins': [3, 5, 7, 10], 
                                },
                         ret_args = {
                             'signal_type': ['signal_quantiles'],
                             'norm_method': ['cdf', 'z-score'],
                             'rebalancing': [1, 3, 5, 7, 10],
                             'weighting': ['ew', 'vol'],
                             't_cost': [0, 0.001, 0.0025, 0.005]
                         })

In [ ]:
strat_param_matrix = param_heatmap(strat_df.dropna(), 
                             metric='sharpe_ratio', 
                             fixed_params={'strategy': 'cs_ls', 'signal_type': 'signal_quantiles', 'rebalancing':7, 'norm_method': 'cdf', 'weighting':'ew'}, 
                             plot_params=['factor_bins', 't_cost']
                                  )